Primeira etapa, extrair os IDs da planilha CSV

In [13]:
import pandas as pd

df = pd.read_csv('clients.csv')
user_ids = df['UserID'].tolist()

print(user_ids)


[2576, 2577, 2580, 2581]


Extrair dados da API REST do Santander Dev Week

In [14]:
import requests #trabalha com http
import json #trabalha com json

#variável com endereço base da api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

#criar uma função para recupeação dos dados
def get_user(id):
  #aponta request como buscar os dados
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  #trata os dados recebidos no retorno da função
  return response.json() if response.status_code == 200 else None

# Compreensão de listas - laço for que varre cada id e vincula
# user ao get_user(id) da função, se seu resultado for válido
# := walrus
users = [user for  id in user_ids if (user := get_user(id)) is not None]

print(json.dumps(users, indent=2))


[
  {
    "id": 2576,
    "name": "Carlos Santos",
    "account": {
      "id": 2714,
      "number": "55555-1",
      "agency": "1111",
      "balance": 3000.0,
      "limit": 500.0
    },
    "card": {
      "id": 2495,
      "number": "**** **** 9012 3456",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2577,
    "name": "Ana Lima",
    "account": {
      "id": 2715,
      "number": "44444-2",
      "agency": "2222",
      "balance": 7000.0,
      "limit": 1200.0
    },
    "card": {
      "id": 2496,
      "number": "**** **** 3456 7890",
      "limit": 1800.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2580,
    "name": "Luiz Souza",
    "account": {
      "id": 2718,
      "number": "77777-3",
      "agency": "3333",
      "balance": 6000.0,
      "limit": 800.0
    },
    "card": {
      "id": 2498,
      "number": "**** **** 7890 1234",
      "limit": 1300.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 2581,
 

Carregamento da IA "Gervásio"

In [1]:
#!pip install transformers

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "PORTULAN/gervasio-ptbr-base"

tokenizer = AutoTokenizer.from_pretrained("PORTULAN/gervasio-ptbr-base")
model = AutoModelForCausalLM.from_pretrained("PORTULAN/gervasio-ptbr-base")



Geração de mensagens



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import random

model_name = "PORTULAN/gervasio-ptbr-base"
tokenizer = AutoTokenizer.from_pretrained("PORTULAN/gervasio-ptbr-base")
model = AutoModelForCausalLM.from_pretrained("PORTULAN/gervasio-ptbr-base")


def generate_ia_news(user):
# Inicializa a lista que irá armazenar as mensagens personalizadas
    mensagens_personalizadas = []

# Loop pelos clientes no banco de dados
    for cliente in clientes:
        nome = cliente["name"]
        saldo = cliente["account"]["balance"]
        limite_credito = cliente["account"]["limit"]

# Gera a modelo de mensagem, com base no saldo e no limite de crédito

    prompts = [
    "Olá, {nome}! Notamos que você possui um saldo de R${saldo:.2f} e um limite de crédito de R${limite_credito:.2f}. Descubra nossas soluções de investimento e maximize seus ganhos!",
    "Caro cliente {nome}, com um saldo de R${saldo:.2f} e um limite de crédito de R${limite_credito:.2f}, você está em uma posição privilegiada para aproveitar ao máximo nossos serviços bancários personalizados. Explore agora!",
    "Prezado {nome}, seu saldo de R${saldo:.2f} e limite de crédito de R${limite_credito:.2f} abrem portas para uma ampla gama de benefícios bancários. Conheça nossos produtos exclusivos e otimize sua experiência conosco."
]

# Escolhe aleatoriamente um prompt
    prompt = random.choice(prompts)

# Substitui as variáveis no prompt pelos valores do cliente
    prompt = prompt.format(nome=nome, saldo=saldo, limite_credito=limite_credito)

# Gera a mensagem personalizada usando o modelo
    input_ids = tokenizer.encode(prompt, return_tensors='pt', padding=False)
    attention_mask = [1] * len(input_ids[0])
    output = model.generate(
        input_ids,
        max_length=200,
        do_sample=False,
        #temperature=0.3,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.tensor([attention_mask])
        )
    mensagem_personalizada = tokenizer.decode(output[0], skip_special_tokens=True)

# Adiciona a mensagem personalizada à lista de mensagens personalizadas

    mensagens_personalizadas.append(mensagem_personalizada)

    return mensagens_personalizadas

clientes = [{'id': 2576, 'name': 'Carlos Santos', 'account': {'id': 2714, 'number': '55555-1', 'agency': '1111', 'balance': 3000.0, 'limit': 500.0}, 'card': {'id': 2495, 'number': '**** **** 9012 3456', 'limit': 1000.0}, 'features': [], 'news': []}, {'id': 2577, 'name': 'Ana Lima', 'account': {'id': 2715, 'number': '44444-2', 'agency': '2222', 'balance': 7000.0, 'limit': 1200.0}, 'card': {'id': 2496, 'number': '**** **** 3456 7890', 'limit': 1800.0}, 'features': [], 'news': []}, {'id': 2580, 'name': 'Luiz Souza', 'account': {'id': 2718, 'number': '77777-3', 'agency': '3333', 'balance': 6000.0, 'limit': 800.0}, 'card': {'id': 2498, 'number': '**** **** 7890 1234', 'limit': 1300.0}, 'features': [], 'news': []}, {'id': 2581, 'name': 'Maria Oliveira', 'account': {'id': 2719, 'number': '98065-4', 'agency': '5432', 'balance': 8000.0, 'limit': 1500.0}, 'card': {'id': 2499, 'number': '**** **** 5678 1234', 'limit': 2500.0}, 'features': [], 'news': []}]

mensagens = generate_ia_news(clientes)
for mensagem in mensagens:
    print(mensagem)





Olá, Maria Oliveira! Notamos que você possui um saldo de R$8000.00 e um limite de crédito de R$1500.00. Descubra nossas soluções de investimento e maximize seus ganhos!
Abraços,
Equipe Investimentos
Responder
Maria Oliveira disse:
13 de julho de 2016 às 16:56
Olá,
Você possui um saldo de R$8000.00 e um limite de crédito de R$1500.00.
Como você está buscando um crédito de R$1500.00, você pode encontrar diversas instituições financeiras que oferecem esse tipo de produto.
Então, se você quer encontrar um crédito de R$1500.00, você pode optar por uma das instituições abaixo


Load (Carregamento) com HTTP PUT

In [5]:
import requests

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

def update_user(user):
    response = requests.put(f'{sdw2023_api_url}/users/{user["id"]}', json=user)
    return True if response.status_code == 200 else False

for client in clientes:
    success = update_user(client)
    print(f'User {client["name"]} updated? {success}!')

User Carlos Santos updated? True!
User Ana Lima updated? True!
User Luiz Souza updated? True!
User Maria Oliveira updated? True!
